In [46]:
import spacy
import gensim
import os
import re
import pandas as pd
from gensim.models import KeyedVectors
from joblib import Parallel, delayed
import itertools
import numpy as np
import pickle

data

In [2]:
%%time
dataset = pd.read_parquet('data/word_embeddings.parquet.gz')
dataset.head()

CPU times: user 37.1 s, sys: 14.6 s, total: 51.7 s
Wall time: 59.6 s


,media,url,timestamp,title,text,tokens,embeddings
0,sports-watch,http://news.livedoor.com/article/detail/5985384/,2011-11-02T09:00:00+0900,石川遼、“異例の交際発表”めぐる水面下の攻防とは？,先月24日深夜、プロゴルファー・石川遼が、マネジメント会社「ケーアイ企画」を通し、一般女性と...,"[先月, 24, 日, 深夜, 、, プロゴルファー, ・, 石川, 遼, が, 、, マネ...","[[-0.071045354, -0.25821245, 0.14480348, -0.22..."
1,sports-watch,http://news.livedoor.com/article/detail/6819620/,2012-08-03T10:00:00+0900,柔道界の現状に、古賀氏「日本の場合は先生の圧力が強い」,ロンドン五輪における柔道男子は、ここまで金メダルを獲得できずにいる。2日も男子100kg級に...,"[ロンドン, 五輪, に, おけ, る, 柔道, 男子, は, 、, ここ, まで, 金, ...","[[-0.09269952, 0.5426098, 0.12492261, -0.18234..."
2,sports-watch,http://news.livedoor.com/article/detail/5571206/,2011-05-20T02:00:00+0900,【Sports Watch】女性店員の契約選手侮辱ツイートでアディダスが謝罪,ヴァンフォーレ甲府のFWで、アディダス契約選手の一人＝ハーフナー・マイクが、18日に入籍を発...,"[ヴァン, フォーレ, 甲府, の, FW, で, 、, アディダス, 契約, 選手, の,...","[[-0.12361497, 0.050540913, -0.46253157, -0.16..."
3,sports-watch,http://news.livedoor.com/article/detail/4970500/,2010-08-27T07:25:00+0900,【Sports Watch】スエマエの五輪後、前田は「人間不信になった感じ」,2008年、北京五輪の舞台で世界ランキング一位のペアから金星を挙げ、一躍その名を知らしめた女...,"[2008, 年, 、, 北京, 五輪, の, 舞台, で, 世界, ランキング, 一, 位...","[[0.19916902, -0.079966575, 0.2989788, -0.0169..."
4,sports-watch,http://news.livedoor.com/article/detail/6158040/,2011-12-30T09:00:00+0900,天皇杯で敗れたチームのファン（？）、チケットを燃やす,29日に行なわれた第91回天皇杯全日本サッカー選手権大会は、J2のFC東京と京都サンガが決勝...,"[29, 日, に, 行なわ, れ, た, 第, 91, 回, 天皇, 杯, 全, 日本, ...","[[0.19836882, -0.106275775, 0.3315136, -0.2118..."


## feature-engineering

In [16]:
dataset.loc[:, 'emb_mean'] = dataset['embeddings'].apply(np.mean)
dataset.head()

,media,url,timestamp,title,text,tokens,embeddings,emb_mean
0,sports-watch,http://news.livedoor.com/article/detail/5985384/,2011-11-02T09:00:00+0900,石川遼、“異例の交際発表”めぐる水面下の攻防とは？,先月24日深夜、プロゴルファー・石川遼が、マネジメント会社「ケーアイ企画」を通し、一般女性と...,"[先月, 24, 日, 深夜, 、, プロゴルファー, ・, 石川, 遼, が, 、, マネ...","[[-0.071045354, -0.25821245, 0.14480348, -0.22...","[0.074116744, -0.06302879, -0.018419443, -0.07..."
1,sports-watch,http://news.livedoor.com/article/detail/6819620/,2012-08-03T10:00:00+0900,柔道界の現状に、古賀氏「日本の場合は先生の圧力が強い」,ロンドン五輪における柔道男子は、ここまで金メダルを獲得できずにいる。2日も男子100kg級に...,"[ロンドン, 五輪, に, おけ, る, 柔道, 男子, は, 、, ここ, まで, 金, ...","[[-0.09269952, 0.5426098, 0.12492261, -0.18234...","[0.100644544, -0.023868158, 0.005920538, -0.07..."
2,sports-watch,http://news.livedoor.com/article/detail/5571206/,2011-05-20T02:00:00+0900,【Sports Watch】女性店員の契約選手侮辱ツイートでアディダスが謝罪,ヴァンフォーレ甲府のFWで、アディダス契約選手の一人＝ハーフナー・マイクが、18日に入籍を発...,"[ヴァン, フォーレ, 甲府, の, FW, で, 、, アディダス, 契約, 選手, の,...","[[-0.12361497, 0.050540913, -0.46253157, -0.16...","[0.1261884, -0.029592466, -0.0027288257, -0.07..."
3,sports-watch,http://news.livedoor.com/article/detail/4970500/,2010-08-27T07:25:00+0900,【Sports Watch】スエマエの五輪後、前田は「人間不信になった感じ」,2008年、北京五輪の舞台で世界ランキング一位のペアから金星を挙げ、一躍その名を知らしめた女...,"[2008, 年, 、, 北京, 五輪, の, 舞台, で, 世界, ランキング, 一, 位...","[[0.19916902, -0.079966575, 0.2989788, -0.0169...","[0.10523655, -0.024903512, -0.0035270725, -0.0..."
4,sports-watch,http://news.livedoor.com/article/detail/6158040/,2011-12-30T09:00:00+0900,天皇杯で敗れたチームのファン（？）、チケットを燃やす,29日に行なわれた第91回天皇杯全日本サッカー選手権大会は、J2のFC東京と京都サンガが決勝...,"[29, 日, に, 行なわ, れ, た, 第, 91, 回, 天皇, 杯, 全, 日本, ...","[[0.19836882, -0.106275775, 0.3315136, -0.2118...","[0.10481595, -0.0038150423, 0.0053625847, -0.0..."


In [48]:
# text と tokens は一致（tokenize による欠損は無し）
re.sub(' ', '', dataset['text'][0]) == dataset['tokens'].apply(lambda x: ''.join(x))[0]

True

In [53]:
# token -> embedding で欠損あり；LSTM など単語シーケンスを受け付けるモデルでの推論においては同品詞単語の平均で補完すべき？
print(len(dataset['tokens'][0]))
print(len(dataset['embeddings'][0]))

540
490


## modeling

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [34]:
X = np.array(dataset['emb_mean'].to_list())
y = dataset['media']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=dataset['media'], random_state=5108)

In [35]:
# check if stratified
pd.concat([y_train.value_counts()/y_train.value_counts().sum(), y_test.value_counts()/y_test.value_counts().sum()], axis=1)

,media,media
sports-watch,0.122194,0.122150
movie-enter,0.118030,0.118350
smax,0.118030,0.118350
dokujo-tsushin,0.118030,0.117807
it-life-hack,0.118030,0.118350
kaden-channel,0.117306,0.117264
peachy,0.114410,0.114007
topic-news,0.104634,0.104235
livedoor-homme,0.069334,0.069490


In [36]:
def train_test(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    return y_pred, y_pred_train

models = [LinearSVC(), SVC(), RandomForestClassifier(), DecisionTreeClassifier()]
results = []
for model in models:
    results.append(train_test(model))
for y_pred, y_pred_train in results:
    print(accuracy_score(y_test, y_pred))
    print(accuracy_score(y_train, y_pred_train), '\n')

0.8713355048859935
0.8973569876900797 

0.8403908794788274
0.8595220854453295 

0.8338762214983714
1.0 

0.6346362649294245
1.0 



In [37]:
for y_pred, y_pred_train in results:
    display(pd.crosstab(y_test, y_pred).style.background_gradient('Blues'))

col_0,dokujo-tsushin,it-life-hack,kaden-channel,livedoor-homme,movie-enter,peachy,smax,sports-watch,topic-news
media,,,,,,,,,
dokujo-tsushin,181,1,2,2,8,18,0,1,4
it-life-hack,3,188,6,0,4,1,7,2,7
kaden-channel,8,11,179,5,3,5,1,0,4
livedoor-homme,8,4,3,95,3,7,1,5,2
movie-enter,5,0,0,0,206,5,0,1,1
peachy,27,1,3,5,13,156,1,2,2
smax,0,5,0,0,1,0,212,0,0
sports-watch,2,0,0,3,0,0,0,213,7
topic-news,1,3,0,3,1,1,0,8,175


col_0,dokujo-tsushin,it-life-hack,kaden-channel,livedoor-homme,movie-enter,peachy,smax,sports-watch,topic-news
media,,,,,,,,,
dokujo-tsushin,176,1,0,5,11,20,0,1,3
it-life-hack,4,186,8,0,6,1,5,2,6
kaden-channel,8,17,164,6,3,7,2,0,9
livedoor-homme,14,8,3,80,4,15,0,3,1
movie-enter,8,0,0,1,205,3,0,0,1
peachy,29,3,3,7,14,151,1,1,1
smax,0,15,0,0,0,2,201,0,0
sports-watch,0,0,0,3,0,0,0,210,12
topic-news,2,4,0,1,1,0,0,9,175


col_0,dokujo-tsushin,it-life-hack,kaden-channel,livedoor-homme,movie-enter,peachy,smax,sports-watch,topic-news
media,,,,,,,,,
dokujo-tsushin,176,0,2,8,12,13,1,1,4
it-life-hack,3,169,14,1,3,7,12,1,8
kaden-channel,5,17,158,8,4,10,4,0,10
livedoor-homme,9,10,2,87,3,14,0,1,2
movie-enter,3,0,1,0,207,4,0,1,2
peachy,23,4,3,7,10,155,3,2,3
smax,0,10,2,0,0,3,203,0,0
sports-watch,0,0,0,2,0,0,0,211,12
topic-news,1,3,5,1,2,0,0,10,170


col_0,dokujo-tsushin,it-life-hack,kaden-channel,livedoor-homme,movie-enter,peachy,smax,sports-watch,topic-news
media,,,,,,,,,
dokujo-tsushin,135,2,3,18,17,27,0,6,9
it-life-hack,6,117,34,15,2,4,29,0,11
kaden-channel,8,25,129,8,4,11,11,2,18
livedoor-homme,15,6,7,64,9,16,2,3,6
movie-enter,10,7,3,9,150,18,3,7,11
peachy,37,7,8,19,13,106,5,5,10
smax,0,20,11,4,3,3,174,1,2
sports-watch,13,2,3,11,5,5,1,163,22
topic-news,5,13,8,7,4,5,3,16,131
